<a href="https://colab.research.google.com/github/arshad831/1-Langchain/blob/main/8_rag_vector_stores_pineconeindexdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone Vector Store

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index llama-index-vector-stores-pinecone

Look at final Project and replicte as assignment
https://huggingface.co/spaces/decodingdatascience/insuranceomantel

In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Creating a Pinecone Index

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Google Colab secrets
openai.api_key = userdata.get('openai')

if openai.api_key:
    os.environ["OPENAI_API_KEY"] = openai.api_key

In [16]:
#PUT Tyour api key

from google.colab import userdata


api_key = userdata.get('PINECONE_API_KEY')

if api_key:
    os.environ["PINECONE_API_KEY"] = api_key


pc = Pinecone(api_key=api_key)

In [20]:
#delete if needed 9 first time no need to delete)
pc.delete_index("quickstart")

In [7]:
# dimensions are for text-embedding-ada-002

pc.create_index(
    name="quickstart",
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


{
    "name": "quickstart",
    "metric": "euclidean",
    "host": "quickstart-0blulnl.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [8]:
pinecone_index = pc.Index("quickstart")

#### Load documents, build the PineconeVectorStore and VectorStoreIndex

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display

Download Data

In [ ]:

#!mkdir -p 'data'
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham_essay.txt'

In [10]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [11]:
documents

[Document(id_='52f4fc58-4753-4705-a627-6623b4ef77c3', embedding=None, metadata={'page_label': '1', 'file_name': 'insurance.pdf', 'file_path': '/content/data/insurance.pdf', 'file_type': 'application/pdf', 'file_size': 1341586, 'creation_date': '2025-09-24', 'last_modified_date': '2025-09-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \nProduct Name: Travel infinity | Product UIN: NBHTGBP22011V012223 \n1. Preamble \nThis ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in \nadvance and the terms, conditions and exclusions of this Policy. Expense incurred outside the p

The previous error occurred because the variable `pinecone_index` was used before it was defined. To fix this, I have combined the code that defines `pinecone_index` and the code that uses it into a single cell.

In [ ]:
# Initialize Pinecone index
pinecone_index = pc.Index("quickstart")

# initialize without metadata filter
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

#### Query Index

May take a minute or so for the index to be ready!

In [13]:
# set Logging to DEBUG for more detailed outputs
import time

query_engine = index.as_query_engine()
start_time = time.time()
response = query_engine.query("what is this document about")
display(Markdown(f"<b>{response}</b>"))

# End timer and print duration
end_time = time.time()
print(f"\nExecution Time: {end_time - start_time:.2f} seconds")

<b>This document is about an insurance policy for travel, specifically detailing various coverage options and conditions related to claims, limitations of liability, other insurance policies, subrogation, alterations to the policy, dispute resolution, and required documentation for making claims related to medical treatment, evacuation, compassionate visits, escort of minor child, and upgradation to business class during hospitalization.</b>


Execution Time: 2.90 seconds


In [21]:
display(Markdown(f"<b>{response}</b>"))

<b>Empty Response</b>

In [14]:
pip install gradio

In [21]:

# --- Imports ---
import os
import logging
import sys
import gradio as gr
from IPython.display import Markdown, display

from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# --- Initialize Pinecone ---

index_name = "quickstart"
dimension = 1536

# Delete index if exists (optional: mirrors original behavior)
if index_name in [idx["name"] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load Data ---
# Create folders & download a sample doc (kept same logic, fixed subfolder creation)
#!mkdir -p 'data/paul_graham'
#!wget -q 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

documents = SimpleDirectoryReader("./data").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# --- System Prompt (polite + answer-from-document constraint) ---
SYSTEM_PROMPT = """You are Aisha, a polite and professional Insurance assistant.
Answer ONLY using the information found in the indexed insurance document(s).
If the answer is not in the document(s), say: "I couldn’t find that in the document."
Keep responses concise, helpful, and courteous.
"""

# --- Query Engine ---
query_engine = index.as_query_engine()

def query_doc(user_question: str):
    if not user_question or not user_question.strip():
        return "Please enter a question."
    full_query = f"""{SYSTEM_PROMPT}

User question:
{user_question.strip()}
"""
    try:
        response = query_engine.query(full_query)
        text = str(response).strip()
        # Gentle post-processing to keep it brief/polite
        return text if text else "I couldn’t find that in the document."
    except Exception as e:
        return f"Error: {str(e)}"

# --- Gradio UI (Professional look with logo, centered title) ---
# Use the raw GitHub URL for proper image rendering.
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/Omantel/main/Omantel_Logo%20(1).png"

CUSTOM_CSS = """
.gradio-container { font-family: Inter, ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, 'Helvetica Neue', Arial; }
.header-wrap {
    display: grid;
    grid-template-columns: 120px 1fr 120px;
    align-items: center;
    gap: 12px;
    padding: 12px 0 8px;
    border-bottom: 1px solid #eaeaea;
}
.header-logo { display:flex; align-items:center; justify-content:flex-start; }
.header-logo img { height: 48px; object-fit: contain; }
.header-title { text-align:center; }
.header-title h1 {
    margin: 0; font-weight: 700; font-size: 1.5rem; line-height: 1.2;
}
.header-spacer { height: 1px; }
.section { padding-top: 8px; }
.footer-note { text-align:center; font-size: 12px; color:#667085; padding: 8px 0 0; }
label.svelte-1ipelgc, .label-wrap label { font-weight: 600; }
"""

with gr.Blocks(css=CUSTOM_CSS, title="Insurance Document QA (LlamaIndex + Pinecone)") as demo:
    # Header with logo (left) and centered title
    with gr.Row(elem_classes="header-wrap"):
        with gr.Column(scale=0, elem_classes="header-logo"):
            gr.HTML(f'<img src="{LOGO_URL}" alt="Omantel Logo" />')
        with gr.Column(scale=1, elem_classes="header-title"):
            gr.HTML("<h1>Insurance QA</h1>")
        with gr.Column(scale=0):
            gr.HTML("")  # right-side spacer

    gr.Markdown(
        "Ask questions based on the Insurance Document "
        "**Answers come only from the document**. If not found, I’ll say so."
    )

    with gr.Group(elem_classes="section"):
        inp = gr.Textbox(
            label="Your question",
            placeholder="e.g., Ask in Insurance Question?",
            lines=2,
        )
        btn = gr.Button("Submit", variant="primary")
        out = gr.Textbox(label="Answer", lines=8)

    btn.click(fn=query_doc, inputs=inp, outputs=out)
    inp.submit(fn=query_doc, inputs=inp, outputs=out)

    gr.Markdown('<div class="footer-note">LlamaIndex + Pinecone • Demo</div>')

demo.launch()


Upserted vectors:   0%|          | 0/61 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9248d5ed1501f4b6ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
documents

[Document(id_='1abfda01-57d9-4c51-9f84-05e64287908c', embedding=None, metadata={'page_label': '1', 'file_name': 'insurance.pdf', 'file_path': '/content/data/insurance.pdf', 'file_type': 'application/pdf', 'file_size': 1341586, 'creation_date': '2025-09-24', 'last_modified_date': '2025-09-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \nProduct Name: Travel infinity | Product UIN: NBHTGBP22011V012223 \n1. Preamble \nThis ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in \nadvance and the terms, conditions and exclusions of this Policy. Expense incurred outside the p